In [3]:
%matplotlib inline

import os
import requests
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import csv
import sys

from pandas.tools.plotting import scatter_matrix

from sklearn import cross_validation as cv
from sklearn.cross_validation import train_test_split as tts
from sklearn.linear_model import Ridge
from sklearn.linear_model import RandomizedLasso
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.cross_validation import cross_val_predict
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import FeatureUnion
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.covariance import EllipticEnvelope
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import IsolationForest

csv.field_size_limit(sys.maxsize)

reviews = "/Users/skhederian/restaurant-health/format_reviews.csv"
data = "/Users/skhederian/restaurant-health/the_final_countdown.csv"

dfr = pd.read_csv(reviews)
dfd = pd.read_csv(data)

//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Predict Violation Bin from Review Text

In [18]:
#Drop NaN Violations
dfreview = dfr.dropna(how = 'any').copy()
dfreview.shape

(20377, 5)

In [38]:
#Load our Pickle Classifier
from sklearn.externals import joblib

filename = 'class.pkl'
clf = joblib.load(filename) 

In [40]:
predict = clf.predict(dfreview['string_agg'].values)
predict.shape

(20377,)

In [44]:
dfreview['predictions'] = predict

In [45]:
dfreview.head()

,business_id,from_date,thru_date,violations,string_agg,predictions
0,--pOlFxITWnhzc7SHSIP0A,2013-04-04,2013-07-29,9.0,My group of four visited tonight for the first...,Perfect
1,--pOlFxITWnhzc7SHSIP0A,2013-07-30,2013-10-29,12.0,My husband and I recently ate dinner here for...,Perfect
2,--pOlFxITWnhzc7SHSIP0A,2013-10-30,2014-03-19,9.0,Our waiter Chris was excellent!!!!!!!!!! My bu...,Perfect
3,--pOlFxITWnhzc7SHSIP0A,2014-03-20,2014-06-11,6.0,I would like to Thank the staff at Block and G...,Excellent
4,--pOlFxITWnhzc7SHSIP0A,2014-06-12,2014-09-17,3.0,Pork Belly appetizer was to die for! A little...,Perfect


# Merge Data and Review DFs

In [51]:
df = dfreview.replace('2999-12-31','2017-01-01')

In [53]:
df.rename(columns = {'--business_id':'_id'}, inplace = True)

In [54]:
df['thru_date'] = pd.to_datetime(df['thru_date'], coerce=True)
dfd['inspection_date'] = pd.to_datetime(dfd['inspection_date'], coerce=True)
df['inspection_date'] = pd.DatetimeIndex(df['thru_date']) + pd.DateOffset(1)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: the coerce=True keyword is deprecated, use errors='coerce' instead
  if __name__ == '__main__':
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: the coerce=True keyword is deprecated, use errors='coerce' instead
  from ipykernel import kernelapp as app


In [55]:
finaldf = pd.merge(dfd, df, on=['business_id', 'inspection_date'], how='inner')

In [57]:
finaldf.shape

(18879, 73)

# Encode Predicted Violations Column

In [59]:
group_names = {'Perfect':5, 'Excellent':10, 'Great':15, 'Good':20, 'Bad':25, 'Very Bad':30, 'rats':35, 'Shutdown':40}

finaldf.predictions = finaldf.predictions.map(group_names)
finaldf.predictions.unique()

array([10,  5, 15, 20, 30, 40])

In [62]:
finaldf.to_csv('pickles.csv')

Now feed this into our regression models!